<a href="https://colab.research.google.com/github/kkmax-develop/FruitDiscrimination/blob/master/Sample_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GPUが割り当てられているかチェック
import tensorflow as tf
tf.test.gpu_device_name()

''

### **ライブラリのインポート**

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

#画像認識関連ライブラリ
from tensorflow import keras
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization, Reshape
from tensorflow.keras.models import Sequential, Model

### **データの前処理**

ImageDataGeneratorクラスのインスタンスを作成。各引数の意味は公式ドキュメントを参照。

https://keras.io/ja/preprocessing/image/

In [3]:
# データ拡張の定義。データ拡張は行わず、正規化とデータ分割のみ実施。trainデータとvalidationデータは8：2で設定。
datagen = image.ImageDataGenerator(rescale=1.0 / 255,validation_split = 0.2)

In [4]:
# ImageDataGeneratorを使用してデータの読み込みと拡張
batch_size = 10000
IMG_WIDTH = 100
IMG_HEIGHT = 100
CHANNELS = 3
CLASSES = 131
train_dir = "/content/drive/My Drive/Colab Notebooks/FruitDiscrimination/data/training"
TARGET = ["Apple Golden 1","Avocado","Banana","Blueberry","Cherry 1","Grape Pink","Kaki",
          
         "Lemon","Mango","Onion Red","Peach","Strawberry","Tomato 1"]

train_generator = datagen.flow_from_directory(
            train_dir,
            target_size=(IMG_WIDTH,IMG_HEIGHT),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=True,
            #classes=TARGET,
            subset = "training" 
        )

Found 54191 images belonging to 131 classes.


In [5]:
# validationデータの読み込み
validation_generator = datagen.flow_from_directory(
            train_dir,
            target_size=(IMG_WIDTH,IMG_HEIGHT),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=True,
            #classes=TARGET,
            subset = "validation" 
        )

Found 13502 images belonging to 131 classes.


In [ ]:
# numpy配列へ変換(CNN等へ読み込ませる場合は不要)
X_train, y_train = train_generator.next()
X_val, y_val = validation_generator.next()

In [ ]:
# 入力画像を行列(100x100x3)からベクトル(長さ30000)に変換
X_train = X_train.reshape(-1, 30000)
X_val = X_val.reshape(-1, 30000)

### **モデル作成**

In [ ]:
# 3層のニューラルネットワークを作成
model = Sequential()

# 最初のlayerはinput_shapeを指定して、入力するデータの次元を与える必要がある
model.add(Dense(units=256, activation='relu', input_shape=(30000,)))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=CLASSES, activation='softmax'))

model.summary()

In [ ]:
# 作成したモデルの可視化
from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True, dpi=None).create(prog='dot', format='svg'))

### **学習**

In [ ]:
# モデルの学習方法について指定
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

epochs = 100

# 学習 
result = model.fit(X_train, y_train,
                   batch_size=128,
                   epochs=epochs,
                   validation_data=(X_val, y_val),)

In [ ]:
result.history.keys()

### **学習状況の可視化**

In [ ]:
plt.style.use('fivethirtyeight')
plt.title('Model Accuracy')
plt.plot(range(1, epochs+1), result.history['accuracy'], label="training")
plt.plot(range(1, epochs+1), result.history['val_accuracy'], label="validation")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.style.use('fivethirtyeight')
plt.title('Model Loss')
plt.plot(range(1, epochs+1), result.history['loss'], label="training")
plt.plot(range(1, epochs+1), result.history['val_loss'], label="validation")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### **モデルの保存**

In [ ]:
model.save('/content/drive/My Drive/Colab Notebooks/FruitDiscrimination/model/BaseModel_NN.h5') 
